<a href="https://colab.research.google.com/github/SamoaChen/Natural_Language_Class_Projects/blob/main/Zhiyang_HW2_CKY_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-----FUNCTIONS

#DEFINE CLASS FOR NODE
class NodeClass:
    def __init__(self, left=None, right1=None, right2=None):
        self.left = left
        self.right = [right1,right2]

#DEFINE FUNCTION THAT RECORDs ALL CNF GRAMMARS
def record_cnf(cnf_file_name):
    
    #INITIATE DICTIONARY TO STORE CNF GRAMMAR
    cnf_grammar_dict = dict()
    
    #LOOP FILE TO STORE GARMMARS
    cnf_file = open(cnf_file_name, 'r')
    for line in cnf_file:
        cnf_words = line.split()
        right_side = cnf_words.copy()
        del right_side[0:2]
        if cnf_words[0] not in cnf_grammar_dict.keys():
            cnf_grammar_dict[cnf_words[0]] = [right_side]
        else:
            cnf_grammar_dict[cnf_words[0]].append(right_side)
    
    cnf_file.close()
    return cnf_grammar_dict


#DEFINE FUNCTION THAT TABULATE PARSE MATRIX
def fill_parse_matrix(cnf_file_name, parse_sentence, cnf_grammar_dict):
    #INITIATE DICTIONARY FOR PARSE MATRIX
    parse_matrix_dict = dict()
    #TABULATE THE PARSE MATRIX
    words = parse_sentence.split()
    #ACTIVATE DICTIONARY SUDICTIONARIES
    for i in range(0,len(words),1):
        parse_matrix_dict[i] = dict()
        
    for j in range(1,len(words)+1,1):
        parse_matrix_dict[j-1][j] = []
        for grammar in cnf_grammar_dict.keys():
            if [words[j-1]] in cnf_grammar_dict[grammar]:
                parse_matrix_dict[j-1][j].append(NodeClass(left=grammar, right1=NodeClass(words[j-1])))


        for i in range(j-2, 0-1, -1):
            parse_matrix_dict[i][j] = []
            for k in range(i+1, (j-1)+1, 1):
                for node_left in parse_matrix_dict[i][k]:
                    for node_right in parse_matrix_dict[k][j]:
                        for grammar in cnf_grammar_dict.keys():
                            if [node_left.left, node_right.left] in cnf_grammar_dict[grammar]:
                                parse_matrix_dict[i][j].append(NodeClass(left=grammar, right1=node_left, right2=node_right))
        
        
        
    return parse_matrix_dict

#DEFINE FUNCTION THAT PRINTS PARSE
def print_parse(print_tree, parse_matrix_dict):
    valid_parse = []
    current_parse_num = 0
    for node in parse_matrix_dict[0][len(parse_matrix_dict.keys())]:
        if node.left == 'S':
            valid_parse.append(node)
    if len(valid_parse) > 0:
        print('VALID SENTENCE')
        #PRINT OUT ALL VALID THREES
        for valid_node in valid_parse:
            current_parse_num += 1
            print('Valid parse #' + str(current_parse_num) + ':')
            print(text_print(valid_node))
            if print_tree:
                print('\n')
                print(tree_print(valid_node,0))
        print('\n')
        print('Number of valid parses: '+str(current_parse_num))
    else:
        print('NO VALID PARSES')

#DEFINE FUNCTION FOR TEXT PRINT
def text_print(valid_node):
    if valid_node.right[1] != None:
        parse_string = '['+valid_node.left+' '+text_print(valid_node.right[0])+' '+text_print(valid_node.right[1])+']' 
    
    else:
        parse_string = '['+valid_node.left+' '+valid_node.right[0].left+']'
    
    return parse_string

#DEFINE FUNCTION FOR TREE PRINT
def tree_print(valid_node, depth):
    if valid_node.right[1] != None:
        parse_string = depth*'   '+'['+valid_node.left+'\n'+tree_print(valid_node.right[0],depth+1)+'\n'+tree_print(valid_node.right[1],depth+1)+'\n'+depth*'   '+']'
    
    else:
        parse_string = depth*'   '+'['+valid_node.left+' '+valid_node.right[0].left+']'
        
    return parse_string
    
#-----PARSE EXECUTION 
if __name__ == "__main__":
    cnf_file_name = input('Enter the file name for the cnf grammar you want to use: ')
    print('Loading grammar...')
    cnf_grammar_dict = record_cnf(cnf_file_name)
    while True:
        print_tree_cond = input('Do you want textual trees to be displayed (y/n)?: ')
        if print_tree_cond == 'y' or print_tree_cond == 'Y':
            print_tree = True
        elif print_tree_cond == 'n' or print_tree_cond == 'N':
            print_tree = False
        parse_sentence = input('Enter a sentence: ')
        if parse_sentence == 'quit':
            print('Goodbye!')
            break
        parse_matrix_dict = fill_parse_matrix(cnf_file_name, parse_sentence, cnf_grammar_dict)
        print_parse(print_tree, parse_matrix_dict)


Enter the file name for the cnf grammar you want to use: SampleGrammar.cnf
Loading grammar...
Do you want textual trees to be displayed (y/n)?: y
Enter a sentence: i boo a flight
NO VALID PARSES
Do you want textual trees to be displayed (y/n)?: y
Enter a sentence: i book a flight
VALID SENTENCE
Valid parse #1:
[S [NP i] [VP [Verb book] [NP [Det a] [Nominal flight]]]]


[S
   [NP i]
   [VP
      [Verb book]
      [NP
         [Det a]
         [Nominal flight]
      ]
   ]
]


Number of valid parses: 1
Do you want textual trees to be displayed (y/n)?: y
Enter a sentence: i book a flight from houston
VALID SENTENCE
Valid parse #1:
[S [NP i] [VP [Verb book] [NP [Det a] [Nominal [Nominal flight] [PP [Preposition from] [NP houston]]]]]]


[S
   [NP i]
   [VP
      [Verb book]
      [NP
         [Det a]
         [Nominal
            [Nominal flight]
            [PP
               [Preposition from]
               [NP houston]
            ]
         ]
      ]
   ]
]
Valid parse #2:
[S [NP i] [